In [13]:
import os
from dotenv import load_dotenv
load_dotenv()

gemini_api_key = os.getenv("GEMINI_API_KEY")
langchain_api_key = os.getenv("LANGCHAIN_API_KEY")
langchain_project = os.getenv("LANGCHAIN_PROJECT")

os.environ["GEMINI_API_KEY"] = gemini_api_key
os.environ["LANGCHAIN_API_KEY"] = langchain_api_key
os.environ["LANGCHAIN_PROJECT"] = langchain_project

In [14]:
from langchain_google_genai import ChatGoogleGenerativeAI
llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash",api_key=gemini_api_key)
llm.invoke('hi').content

'Hi there! How can I help you today?'

In [15]:
message = [{"role":"system","content":"you are a helpful assistant"},
           {"role":"user","content":"hi how are you?"}]

result = llm.invoke(message)
print(result.content)

I am doing well, thank you for asking! How are you today?


In [16]:
class Chatbot:
    def __init__(self, system=""):
        self.system = system
        self.message = []
        if self.system:
            self.message.append({"role":"system","content":system})

    def __call__(self, message):
        self.message.append({"role":"user","content":message})
        result = self.execute()
        self.message.append({"role":"assistant","content":result})
        return result
    
    def execute(self):
        llm= ChatGoogleGenerativeAI(model="gemini-2.0-flash",api_key=gemini_api_key)
        result = llm.invoke(self.message)
        return result.content

In [17]:
bot = Chatbot("you are a helpful assistant")
bot("HI")

'Hi there! How can I help you today?'

In [18]:
bot.message

[{'role': 'system', 'content': 'you are a helpful assistant'},
 {'role': 'user', 'content': 'HI'},
 {'role': 'assistant', 'content': 'Hi there! How can I help you today?'}]

In [19]:
bot.execute()

''

In [20]:
prompt = """
You run in a loop of Thought, Action, PAUSE, Observation.
At the end of the loop your output an Answer.
Use Thought to describe your thoughts about the question you have been asked.
Use Action to run one of the actions available to you - then return PAUSE.
Observation will be the result of running those actions.


Your available actions are:
calculate:
e.g. calculate: 4 * 7 / 3
Runs a calculation and returns the number - uses Python so be sure to use floating point
syntax if necessary

wikipedia:
e.g. wikipedia: Django
Returns a summary from searching Wikipedia

simon_blog_search:
e.g. simon_blog_search: Python
Search Simon's blog for that term

Example session:
Question: What is the capital of France?
Thought: I should look up France on Wikipedia
Action: wikipedia: France
PAUSE

You will be called again with this:
Observation: France is a country. The capital is Paris.

You then output:
Answer: The capital of France is Paris

Please Note: if you get basic conversation questions like "hi","hello","how are you?",\n
you have to answer "hi","hello","i am good".
""".strip()

Understand the Regular expression
Pattern Breakdown:

^: This matches the start of a string. It means the string must begin with what follows.
Action:: This is a literal match. It means the string must have the text "Action:" at the beginning.
(\w+):
The parentheses () define a capture group. This allows you to extract part of the string that matches this section.
\w+ matches one or more word characters (letters, digits, and underscores). This will capture a word that follows "Action:".
:: This is a literal colon that separates the word matched by (\w+) from the rest of the string.
(.*):
This is another capture group, where .* matches any character (.) zero or more times (*), which means it captures everything that comes after the second colon.
What does it do?
This regex is looking for a string that:

Starts with the text "Action:".
Has a word right after it, separated by a colon.
Then, after another colon, it captures everything that follows.

In [24]:
import re
action_re = re.compile(r'^Action: (\w+): (.*)')

In [25]:
import re

# Compiling the regular expression pattern
action_re = re.compile(r'^Action: (\w+): (.*)')

# Sample strings
text1 = "Action: Move: North"
text2 = "Action: Jump: High"
text3 = "Error: Not an Action"

In [27]:
match1 = action_re.match(text1)
match1

<re.Match object; span=(0, 19), match='Action: Move: North'>

In [34]:
match1.group(1)


'Move'

In [35]:
match1.group(2)

'North'

In [31]:
match2 = action_re.match(text2)
match2

<re.Match object; span=(0, 18), match='Action: Jump: High'>

In [30]:
match3 = action_re.match(text3)
match3

In [32]:
if match1:
    print(f"Matched: {match1.group(1)}, {match1.group(2)}")

Matched: Move, North


In [36]:
if match2:
    print(f"Matched: {match2.group(1)}, {match2.group(2)}")  # Jump, High

Matched: Jump, High


In [37]:
if match3:
    print("No match")  # This won't match, as it doesn't start with "Action:"

In [38]:
import httpx
def wikipedia(query):
    response = httpx.get("https://en.wikipedia.org/w/api.php", params={
        "action": "query",
        "list": "search",
        "srsearch": query,
        "format": "json"
    })
    return response.json()["query"]["search"][0]["snippet"]

In [39]:
import httpx
def simon_blog_search(query):
    response = httpx.get("https://datasette.simonwillison.net/simonwillisonblog.json", params={
        "sql": """
        select
          blog_entry.title || ': ' || substr(html_strip_tags(blog_entry.body), 0, 1000) as text,
          blog_entry.created
        from
          blog_entry join blog_entry_fts on blog_entry.rowid = blog_entry_fts.rowid
        where
          blog_entry_fts match escape_fts(:q)
        order by
          blog_entry_fts.rank
        limit
          1
        """.strip(),
        "_shape": "array",
        "q": query,
    })
    return response.json()[0]["text"]

In [40]:
def calculate(number):
    return eval(number)

In [41]:
calculate("2+2")

4

In [48]:
known_actions = {
    "wikipedia": wikipedia,
    "calculate": calculate,
    "simon_blog_search": simon_blog_search
}

In [49]:
bot = Chatbot(prompt)

In [50]:
next_prompt="Tell me about quantum computing from Wikipedia."

result = bot(next_prompt)

result

'Thought: I need to search Wikipedia for information about quantum computing.\nAction: wikipedia: quantum computing\nPAUSE'

In [51]:
actions = [action_re.match(a) for a in result.split('\n') if action_re.match(a)]

actions

[<re.Match object; span=(0, 36), match='Action: wikipedia: quantum computing'>]

In [52]:
action, action_input = actions[0].groups()

action_input

'quantum computing'

In [53]:
action

'wikipedia'

In [54]:
known_actions[action](action_input)

'and <span class="searchmatch">quantum</span> <span class="searchmatch">computing</span> takes advantage of this behavior using specialized hardware. Classical physics cannot explain the operation of these <span class="searchmatch">quantum</span> devices'

In [57]:
def query(question, max_turns=5):
    i=0
    bot = Chatbot(prompt)
    next_prompt = question
    while i < max_turns:
        i +=1 
        result = bot(next_prompt)
        print(result)
        actions = [action_re.match(a) for a in result.split('\n') if action_re.match(a)]
        if actions:
            action, action_input = actions[0].groups()
            if action not in known_actions:
                raise Exception(f"Unknown action: {action}: {action_input}")
            print(" -- running {} {}".format(action, action_input))
            observation = known_actions[action](action_input)
            print("Observation:", observation)
            next_prompt = f"Observation: {observation}"

        else:
            return result

In [58]:
query("hello")

Thought: This is a basic conversational question.
Action: PAUSE
Observation:
Answer: hi


'Thought: This is a basic conversational question.\nAction: PAUSE\nObservation:\nAnswer: hi'

In [59]:
query("what you doing?")

Thought: This is a basic conversation question. I should respond appropriately.
Action:
PAUSE
Observation:
Answer: hi


'Thought: This is a basic conversation question. I should respond appropriately.\nAction:\nPAUSE\nObservation:\nAnswer: hi'

In [60]:
print(query("Fifteen * twenty five"))

Thought: I need to calculate the product of 15 and 25.
Action: calculate: 15 * 25
PAUSE
 -- running calculate 15 * 25
Observation: 375
Answer: 375
Answer: 375


In [61]:
query("Has Simon written about AI?")

Thought: I need to search Simon's blog for the term "AI" or "Artificial Intelligence" to see if he has written about it.
Action: simon_blog_search: AI
PAUSE
 -- running simon_blog_search AI
Observation: It's OK to call it Artificial Intelligence: Update 9th January 2024: This post was clumsily written and failed to make the point I wanted it to make. I've published a follow-up, What I should have said about the term Artificial Intelligence which you should read instead.
My original post follows.


We need to be having high quality conversations about AI: what it can and can't do, its many risks and pitfalls and how to integrate it into society in the most beneficial ways possible.
Any time I write anything that mentions AI it's inevitable that someone will object to the very usage of the term.
Strawman: "Don't call it AI! It's not actually intelligent - it's just spicy autocomplete."
That strawman is right: it's not "intelligent" in the same way that humans are. And "spicy autocomplete

'Thought: Simon has written about AI and the use of the term "Artificial Intelligence".\nAnswer: Yes, Simon has written about AI, specifically regarding the use of the term "Artificial Intelligence".'